In [1]:
import pandas as pd
import librosa

import os
PATH = os.getenv('AUDIO_PATH')

In [3]:
PATH

In [2]:
audio_PATH = PATH + "fma_small/"

audio = []
rates = []
track_id = []

bad_files = [
    '098/098565.mp3',
    '098/098567.mp3',
    '098/098569.mp3',
    '099/099134.mp3',
    '108/108925.mp3',
    '133/133297.mp3'
]


for item in os.listdir(audio_PATH):
    if 'check' not in item and '.txt' not in item:
        for file in os.listdir(audio_PATH + item):
            if item + '/' + file in bad_files:
                continue
            track_id.append(item + '/' + file)
            file_path = audio_PATH + track_id[-1]

            song, sr = librosa.load(file_path)

            mel = librosa.power_to_db(librosa.feature.melspectrogram(y = song, sr = sr)).T[:1291]

            audio.append(mel)
            rates.append(sr)

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [ ]:
raw_data = pd.Series(
    audio, 
    index = pd.Series(
        [int(id[-10:-4]) for id in track_id]
        ), 
    name = 'audio')

In [ ]:
df = pd.read_csv(PATH + "fma_metadata/tracks.csv", index_col=0, header=[0, 1])

In [ ]:
artist_album = pd.merge(
    df['artist'],
    df['album'],
    how = 'inner',
    left_index = True,
    right_index = True
).rename(
    {
        'name' : 'name',
        'title' : 'album'
    },
    axis = 1
)[['name', 'album']]

genre_artists = pd.merge(
    df['set'][df['set']['subset'] == 'small'],
    artist_album,
    how = 'left',
    left_index = True,
    right_index = True
)

tracks = pd.merge(
    df['track'],
    genre_artists,
    how = 'inner',
    left_index = True,
    right_index = True
)[['title', 'name', 'album', 'genre_top', 'split']]


indices = [int(file[-10:-4]) for file in bad_files]
# tracks.loc[~tracks.index.isin(indices), ].to_csv('../data/tracks_meta.csv')

In [ ]:
for idx, data in zip(raw_data.index, raw_data.values):
    # np.save('../data/audio' + str(idx), data)
    pass